In [1]:
# Boston_Dataset_Deep_Neural_Net_과제답안
# 3 Layer DNN(Deep Neural Net)

import tensorflow as tf
import numpy as np
import csv
tf.random.set_seed(5)

# 구글 코랩 사용 시 구글 드라이브 마운트 수행코드
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# 구글 코랩 사용 시 구글 드라이브 경로 지정, 사용자 디렉토리가 'My Drive'일 경우
# boston_train = np.loadtxt('/content/drive/My Drive/boston_train.csv', delimiter=',', dtype=np.float32, skiprows=1)
# boston_test = np.loadtxt('/content/drive/My Drive/boston_test.csv', delimiter=',', dtype=np.float32, skiprows=1)

boston_train = np.loadtxt('boston_train.csv', delimiter=',', dtype=np.float32, skiprows=1)
boston_test = np.loadtxt('boston_test.csv', delimiter=',', dtype=np.float32, skiprows=1)

x_train = boston_train[:,:9]
y_train = boston_train[:,[-1]]
print(x_train.shape)   # (400,9)
print(y_train.shape)   # (400,1)

(400, 9)
(400, 1)


In [3]:
# Layer 1
# (400,9) * (9,5) = (400,5)
W1 = tf.Variable(tf.random.normal([9,5]), name='weight1')
b1 = tf.Variable(tf.random.normal([5]), name='bias1')

@tf.function
def layer1(X):
    return  tf.nn.relu(tf.matmul(X,W1) + b1 )

In [4]:
# layer 2
# (400,5) * (5,2) = (400,2)
W2 = tf.Variable(tf.random.normal([5,2]), name='weight2')
b2 = tf.Variable(tf.random.normal([2]), name='bias2')

@tf.function
def layer2(X):
    return  tf.nn.relu(tf.matmul(layer1(X),W2) + b2 )

In [5]:
# layer 3
# (400,2) * (2,1) = (400,1)
W3 = tf.Variable(tf.random.normal([2,1]), name='weight3')
b3 = tf.Variable(tf.random.normal([1]), name='bias3')

In [6]:
# hypothesis 예측 함수(방정식)  , H(x) = W * X + b
@tf.function
def hypothesis(X):
    return  tf.matmul(layer2(X),W3) + b3  

In [7]:
# 비용함수 : (Hx - y)^2 의 평균
@tf.function
def cost_func():
    cost = tf.reduce_mean(tf.square((hypothesis(x_train) - y_train)))
    return cost

In [8]:
# 경사 하강법
# learning rate (학습율) 을 0.01로 설정하여  optimizer객체를 생성
# optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=0.01)
optimizer = tf.keras.optimizers.Adam(lr=0.01)

In [9]:
# 학습 시작
print('***** Start Learning!!')
for step in range(10001):
    # cost를 minimize한다
    optimizer.minimize(cost_func, var_list=[W1,b1,W2,b2,W3,b3])
    
    if step % 1000 == 0:
        print('%04d'%step, 'cost: [', cost_func().numpy(), ']' )
        
print('***** Learning Finished!!')

***** Start Learning!!
0000 cost: [ 642.07025 ]
1000 cost: [ 48.002228 ]
2000 cost: [ 29.871864 ]
3000 cost: [ 29.093866 ]
4000 cost: [ 28.794493 ]
5000 cost: [ 28.708157 ]
6000 cost: [ 28.683994 ]
7000 cost: [ 28.708096 ]
8000 cost: [ 28.658808 ]
9000 cost: [ 28.639614 ]
10000 cost: [ 28.66671 ]
***** Learning Finished!!


In [10]:
# 회귀 계수 출력
print('Weight =', W1.numpy())
print('Bias =', b1.numpy())

Weight = [[-1.8030666e-01 -1.1792762e+00 -3.9640494e-02 -4.1436806e-01
   1.3231523e+00]
 [-6.1854804e-01  4.1731346e-01 -8.8999532e-02  1.3872666e-02
   7.6250798e-01]
 [ 1.2659615e+00  6.3132334e-01  1.5293121e+00 -1.2260184e-03
   8.9870441e-01]
 [-1.2508010e+00 -1.2747028e+00  1.2607460e+00 -1.7003698e+01
   2.6157657e-02]
 [ 2.2328745e-01  5.2540141e-01 -3.7048063e-01  8.7403126e+00
   1.3794525e-01]
 [ 7.5969249e-01  8.1761904e-02 -1.3800851e+00 -5.3248797e-02
   1.1383884e+00]
 [-9.1330576e-01  2.8136900e-01 -6.7893195e-01 -1.4076461e+00
  -4.9823263e-01]
 [-9.3361068e-01 -3.8816217e-01 -4.0501364e-02 -9.6510549e-04
  -1.4186655e+00]
 [-9.0990627e-01  7.4242425e-01 -1.1188040e-01 -9.1340071e-01
  -1.4201527e+00]]
Bias = [ 0.22652863  0.37694615  0.7466094  -9.805608    0.72448725]


In [11]:
# 예측
x_test = boston_test[:,:9]
y_test = boston_test[:,[-1]]

preds = hypothesis(x_test).numpy()

In [12]:
# 정확도 측정(RMSE)
def get_rmse(y_test,preds):
    squared_error = 0
    for k,_ in enumerate(y_test):
        squared_error += (preds[k] - y_test[k])**2
    mse = squared_error/len(y_test)    
    rmse = np.sqrt(mse)
    return rmse[0]  

print('RMSE:', get_rmse(y_test,preds))
# layer 1 ---->   RMSE: 4.0023303 
# layer 3 ---->   RMSE: 3.6358266  , RMSE가 감소하여 정확도가 향상됨

RMSE: 3.6358266
